## Amazon SageMaker Awesome Builder Notebook

We will use Amazon SageMaker Processing jobs to leverage a simplified, managed experience to run data pre- or post-processing and model evaluation workloads on the Amazon SageMaker platform.

A processing job downloads input from Amazon Simple Storage Service (Amazon S3), then uploads outputs to Amazon S3 during or after the processing job.

This notebook is used to run a processing job using a scikit-learn script that cleans, pre-processes, performs feature engineering, and splits the input data into train and test sets.

The dataset loaded here is from PAMAP2 Physical Activity Dataset from UCI database:
https://archive.ics.uci.edu/ml/datasets/PAMAP2+Physical+Activity+Monitoring

We will select features from this dataset, clean the data, and turn the data into features that the training algorithm can use to train a multi-class classification model, and split the data into train and test sets. 

After training a logistic regression model, you evaluate the model against a hold-out test dataset, and save the classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.

## Data pre-processing and feature engineering

To run the scikit-learn preprocessing script as a processing job, create a `SKLearnProcessor`, which lets you run scripts inside of processing jobs using the scikit-learn image provided.

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1)

print('Region: {}'.format(region))
print('Role: {}'.format(role))

Region: us-east-1
Role: arn:aws:iam::572539092864:role/service-role/AmazonSageMaker-ExecutionRole-20200407T174741


Before writing the script we will use for data cleaning, pre-processing, and feature engineering, we inspect the first 10 rows (observations) of the dataset. 

The initial label for the observation is the `activity_id` category. The primary features from the dataset you select are `heart_rate`, `imu_wrist_temp` (temperature in Celsius), and 3 sets of Inertial Measurement Units (IMU) data. IMU data is collected from 3 separate sensors located on wearable devices, like a Smart Watch, plus a heart rate monitor. 

The dataset can be used for activity recognition and intensity estimation, while developing and applying algorithms of data processing, segmentation, feature extraction and classification.

** Sensors **
3 Colibri wireless inertial measurement units (IMU):
- sampling frequency: 100Hz
- position of the sensors:
- 1 IMU over the wrist on the dominant arm
- 1 IMU on the chest
- 1 IMU on the dominant side's ankle
HR-monitor:
- sampling frequency: ~9Hz

In [2]:
import pandas as pd
import numpy as np

bucket_name = 'octank-smartwatch-data'
s3prefix = 'sagemaker-train'

input_data_uri = 's3://{}/{}/subject101.csv'.format(bucket_name, s3prefix)

print('S3 input URI: {}'.format(input_data_uri))

S3 input URI: s3://octank-smartwatch-data/sagemaker-train/subject101.csv


In [3]:
column_names = ['timestamp', 'activity_id', 'heart_rate', 'imu_wrist_temp', 
  'imu_wrist_accel16_x', 'imu_wrist_accel16_y', 'imu_wrist_accel16_z', 
  'imu_wrist_accel6_x', 'imu_wrist_accel6_y', 'imu_wrist_accel6_z', 
  'imu_wrist_gyro_x', 'imu_wrist_gyro_y', 'imu_wrist_gyro_z', 
  'imu_wrist_magnet_x', 'imu_wrist_magnet_y', 'imu_wrist_magnet_z', 
  'imu_wrist_orient1', 'imu_wrist_orient2', 'imu_wrist_orient3', 'imu_wrist_orient4', 
  'imu_chest_temp', 'imu_chest_accel16_x', 'imu_chest_accel16_y', 'imu_chest_accel16_z', 
  'imu_chest_accel6_x', 'imu_chest_accel6_y', 'imu_chest_accel6_z', 
  'imu_chest_gyro_x', 'imu_chest_gyro_y', 'imu_chest_gyro_z', 
  'imu_chest_magnet_x', 'imu_chest_magnet_y', 'imu_chest_magnet_z', 
  'imu_chest_orient1', 'imu_chest_orient2', 'imu_chest_orient3', 'imu_chest_orient4', 
  'imu_ankle_temp', 'imu_ankle_accel16_x', 'imu_ankle_accel16_y', 'imu_ankle_accel16_z', 
  'imu_ankle_accel6_x', 'imu_ankle_accel6_y', 'imu_ankle_accel6_z', 
  'imu_ankle_gyro_x', 'imu_ankle_gyro_y', 'imu_ankle_gyro_z', 
  'imu_ankle_magnet_x', 'imu_ankle_magnet_y', 'imu_ankle_magnet_z', 
  'imu_ankle_orient1', 'imu_ankle_orient2', 'imu_ankle_orient3', 'imu_ankle_orient4']

df = pd.read_csv(input_data_uri, header=0, names=column_names, index_col=False)

In [4]:
# inspect DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376416 entries, 0 to 376415
Data columns (total 54 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   timestamp            376416 non-null  float64
 1   activity_id          376416 non-null  int64  
 2   heart_rate           34388 non-null   float64
 3   imu_wrist_temp       374962 non-null  float64
 4   imu_wrist_accel16_x  374962 non-null  float64
 5   imu_wrist_accel16_y  374962 non-null  float64
 6   imu_wrist_accel16_z  374962 non-null  float64
 7   imu_wrist_accel6_x   374962 non-null  float64
 8   imu_wrist_accel6_y   374962 non-null  float64
 9   imu_wrist_accel6_z   374962 non-null  float64
 10  imu_wrist_gyro_x     374962 non-null  float64
 11  imu_wrist_gyro_y     374962 non-null  float64
 12  imu_wrist_gyro_z     374962 non-null  float64
 13  imu_wrist_magnet_x   374962 non-null  float64
 14  imu_wrist_magnet_y   374962 non-null  float64
 15  imu_wrist_magnet_

In [5]:
# Viewing first few rows of data, we see that 'heart_rate' column is sparse with lots of NaN
# this is expected since the HR sensor collects data at 9Hz vs. 100Hz for the other metrics
print(df.shape)
df.head(n=15)

(376416, 54)


,timestamp,activity_id,heart_rate,imu_wrist_temp,imu_wrist_accel16_x,imu_wrist_accel16_y,imu_wrist_accel16_z,imu_wrist_accel6_x,imu_wrist_accel6_y,imu_wrist_accel6_z,...,imu_ankle_gyro_x,imu_ankle_gyro_y,imu_ankle_gyro_z,imu_ankle_magnet_x,imu_ankle_magnet_y,imu_ankle_magnet_z,imu_ankle_orient1,imu_ankle_orient2,imu_ankle_orient3,imu_ankle_orient4
0,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0
1,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0
2,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0
3,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0
4,8.43,0,NaN,30.0,2.29959,8.82929,3.54710,2.26132,8.65762,3.77788,...,-0.006089,-0.016024,0.001050,-60.2954,-38.8778,-58.3977,1.0,0.0,0.0,0.0
5,8.44,0,NaN,30.0,2.33738,8.82900,3.54767,2.27703,8.77828,3.73230,...,-0.031973,-0.053934,0.015594,-60.6307,-38.8676,-58.2711,1.0,0.0,0.0,0.0
6,8.45,0,NaN,30.0,2.37142,9.05500,3.39347,2.39786,8.89814,3.64131,...,-0.019643,-0.039937,-0.000785,-60.5171,-38.9819,-58.2733,1.0,0.0,0.0,0.0
7,8.46,0,NaN,30.0,2.33951,9.13251,3.54668,2.44371,8.98841,3.62596,...,0.013747,-0.010042,0.017701,-61.2916,-39.6182,-58.1499,1.0,0.0,0.0,0.0
8,8.47,0,NaN,30.0,2.25966,9.09415,3.43015,2.42877,9.01871,3.61081,...,0.007649,-0.013923,0.014498,-60.8509,-39.0821,-58.1478,1.0,0.0,0.0,0.0
9,8.48,0,104.0,30.0,2.29745,8.90450,3.46984,2.39736,8.94335,3.53551,...,0.078900,0.002283,0.020352,-61.5302,-38.7240,-58.3860,1.0,0.0,0.0,0.0


### Here are the raw labels, activity_ids, coming in from the dataset

List of activityIDs and corresponding activities:

[
0=other (transient activities),
1=lying,
2=sitting,
3=standing,
4=walking,
5=running,
6=cycling,
7=Nordic walking,
9=watching TV,
10=computer work,
11=car driving,
12=ascending stairs,
13=descending stairs,
16=vacuum cleaning,
17=ironing,
18=folding laundry,
19=house cleaning,
20=playing soccer,
24=rope jumping
]

In [6]:
# Note: unless we impute the heart_rate data, we will lose most of our dataset to 'NaN' dropping
print('Number of elements BEFORE drop: {}'.format(df.size))

df_drop_remaining = df.dropna(inplace=False)
print('Number of elements AFTER drop: {}'.format(df_drop_remaining.size))
print('Percentage of dataset remaining: {:.2f} %'.format(100*(df_drop_remaining.size/df.size)))

Number of elements BEFORE drop: 20326464
Number of elements AFTER drop: 1840806
Percentage of dataset remaining: 9.06 %


In [7]:
# Let's first remove the heart rate column ('heart_rate') from data frame
# we can then impute the column by applying backfill to the column

df_heart_rate_column = df.pop('heart_rate')
df_heart_rate_column.fillna(value=None, method='backfill', axis=0, inplace=True)

In [8]:
# Insert backfilled heart rate column back into the data frame
df.insert(2, 'heart_rate', df_heart_rate_column)

In [9]:
# shapes
print(df.shape)
print(df_heart_rate_column.shape)

(376416, 54)
(376416,)


In [10]:
# Now, 'heart_rate' column added back to data frame with backfilled data
print(df.shape)

(376416, 54)


In [11]:
# Ah, that looks much better
df.head(n=15)

,timestamp,activity_id,heart_rate,imu_wrist_temp,imu_wrist_accel16_x,imu_wrist_accel16_y,imu_wrist_accel16_z,imu_wrist_accel6_x,imu_wrist_accel6_y,imu_wrist_accel6_z,...,imu_ankle_gyro_x,imu_ankle_gyro_y,imu_ankle_gyro_z,imu_ankle_magnet_x,imu_ankle_magnet_y,imu_ankle_magnet_z,imu_ankle_orient1,imu_ankle_orient2,imu_ankle_orient3,imu_ankle_orient4
0,8.39,0,104.0,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0
1,8.40,0,104.0,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0
2,8.41,0,104.0,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0
3,8.42,0,104.0,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0
4,8.43,0,104.0,30.0,2.29959,8.82929,3.54710,2.26132,8.65762,3.77788,...,-0.006089,-0.016024,0.001050,-60.2954,-38.8778,-58.3977,1.0,0.0,0.0,0.0
5,8.44,0,104.0,30.0,2.33738,8.82900,3.54767,2.27703,8.77828,3.73230,...,-0.031973,-0.053934,0.015594,-60.6307,-38.8676,-58.2711,1.0,0.0,0.0,0.0
6,8.45,0,104.0,30.0,2.37142,9.05500,3.39347,2.39786,8.89814,3.64131,...,-0.019643,-0.039937,-0.000785,-60.5171,-38.9819,-58.2733,1.0,0.0,0.0,0.0
7,8.46,0,104.0,30.0,2.33951,9.13251,3.54668,2.44371,8.98841,3.62596,...,0.013747,-0.010042,0.017701,-61.2916,-39.6182,-58.1499,1.0,0.0,0.0,0.0
8,8.47,0,104.0,30.0,2.25966,9.09415,3.43015,2.42877,9.01871,3.61081,...,0.007649,-0.013923,0.014498,-60.8509,-39.0821,-58.1478,1.0,0.0,0.0,0.0
9,8.48,0,104.0,30.0,2.29745,8.90450,3.46984,2.39736,8.94335,3.53551,...,0.078900,0.002283,0.020352,-61.5302,-38.7240,-58.3860,1.0,0.0,0.0,0.0


In [12]:
# We can replace the raw activity_id (integers) with their label meanings

raw_activity_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

activity_labels = ['unassigned', 'lying', 'sitting', 'standing ', 'walking', 'running', 'cycling', 
                'nordic_walking ', 'missing_8', 'watching_tv ', 'computer_work ', 'car_driving ', 
                'ascending_stairs ', 'descending_stairs ', 'missing_14', 'missing_15', 
                'vacuuming', 'ironing', 'folding_laundry', 'house_cleaning', 'playing soccer',
                'missing_21', 'missing_22', 'missing_23', 'rope jumping'] 

df_with_labels = df.replace(raw_activity_ids, activity_labels, inplace=False)

In [13]:
# Inspect quality of 'activity_id' making sure no out-of-bounds data
for idx in range(0,100000):
    row = df_with_labels.iloc[idx]
    if (row['activity_id'] not in activity_labels):
        print('bad value in row[{}]: {}'.format(idx, row['activity_id']))

In [14]:
# Let's take a look at 'activity_id' label and show each transition in the first 100K rows
prev_activity = None
for idx in range(0,100000):
    row = df_with_labels.iloc[idx]
    if (row['activity_id'] != prev_activity):
        print('Transition to new activity label[{}]: {}'.format(idx, row['activity_id']))
        prev_activity = row['activity_id']

Transition to new activity label[0]: unassigned
Transition to new activity label[2927]: lying
Transition to new activity label[30114]: sitting
Transition to new activity label[53594]: standing 
Transition to new activity label[75311]: unassigned
Transition to new activity label[84966]: ironing


In [15]:
# Let's get the distinct list of unique activities
print(np.unique(df_with_labels['activity_id'].to_numpy()))

['ascending_stairs ' 'cycling' 'descending_stairs ' 'ironing' 'lying'
 'nordic_walking ' 'rope jumping' 'running' 'sitting' 'standing '
 'unassigned' 'vacuuming' 'walking']


In [16]:
# Let's find out how many unique/discrete values per column in our dataset
df.nunique(axis='index')

timestamp              376416
activity_id                13
heart_rate                106
imu_wrist_temp             63
imu_wrist_accel16_x    293671
imu_wrist_accel16_y    286012
imu_wrist_accel16_z    285561
imu_wrist_accel6_x     301814
imu_wrist_accel6_y     294254
imu_wrist_accel6_z     289902
imu_wrist_gyro_x       351873
imu_wrist_gyro_y       350990
imu_wrist_gyro_z       350869
imu_wrist_magnet_x     319042
imu_wrist_magnet_y     290127
imu_wrist_magnet_z     312864
imu_wrist_orient1           1
imu_wrist_orient2           1
imu_wrist_orient3           1
imu_wrist_orient4           1
imu_chest_temp             88
imu_chest_accel16_x    255269
imu_chest_accel16_y    214062
imu_chest_accel16_z    242003
imu_chest_accel6_x     312255
imu_chest_accel6_y     218291
imu_chest_accel6_z     275707
imu_chest_gyro_x       350563
imu_chest_gyro_y       353162
imu_chest_gyro_z       349141
imu_chest_magnet_x     306129
imu_chest_magnet_y     250008
imu_chest_magnet_z     307339
imu_chest_

This notebook cell writes a file `preprocessing.py`, which contains the pre-processing script. You can update the script, and rerun this cell to overwrite `preprocessing.py`. You run this as a processing job in the next cell. In this script, you

* Remove duplicates and rows with conflicting data
* transform the target `income` column into a column containing two labels.
* transform the `age` and `num persons worked for employer` numerical columns into categorical features by binning them
* scale the continuous `capital gains`, `capital losses`, and `dividends from stocks` so they're suitable for training
* encode the `education`, `major industry code`, `class of worker` so they're suitable for training
* split the data into training and test datasets, and saves the training features and labels and test features and labels.

Our training script will use the pre-processed training features and labels to train a model, and our model evaluation script will use the trained model and pre-processed test features and labels to evaluate the model.

In [17]:
%%writefile preprocessing.py

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

column_names = ['timestamp', 'activity_id', 'heart_rate', 'imu_wrist_temp', 
  'imu_wrist_accel16_x', 'imu_wrist_accel16_y', 'imu_wrist_accel16_z', 
  'imu_wrist_accel6_x', 'imu_wrist_accel6_y', 'imu_wrist_accel6_z', 
  'imu_wrist_gyro_x', 'imu_wrist_gyro_y', 'imu_wrist_gyro_z', 
  'imu_wrist_magnet_x', 'imu_wrist_magnet_y', 'imu_wrist_magnet_z', 
  'imu_wrist_orient1', 'imu_wrist_orient2', 'imu_wrist_orient3', 'imu_wrist_orient4', 
  'imu_chest_temp', 'imu_chest_accel16_x', 'imu_chest_accel16_y', 'imu_chest_accel16_z', 
  'imu_chest_accel6_x', 'imu_chest_accel6_y', 'imu_chest_accel6_z', 
  'imu_chest_gyro_x', 'imu_chest_gyro_y', 'imu_chest_gyro_z', 
  'imu_chest_magnet_x', 'imu_chest_magnet_y', 'imu_chest_magnet_z', 
  'imu_chest_orient1', 'imu_chest_orient2', 'imu_chest_orient3', 'imu_chest_orient4', 
  'imu_ankle_temp', 'imu_ankle_accel16_x', 'imu_ankle_accel16_y', 'imu_ankle_accel16_z', 
  'imu_ankle_accel6_x', 'imu_ankle_accel6_y', 'imu_ankle_accel6_z', 
  'imu_ankle_gyro_x', 'imu_ankle_gyro_y', 'imu_ankle_gyro_z', 
  'imu_ankle_magnet_x', 'imu_ankle_magnet_y', 'imu_ankle_magnet_z', 
  'imu_ankle_orient1', 'imu_ankle_orient2', 'imu_ankle_orient3', 'imu_ankle_orient4']


raw_activity_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

activity_labels = ['unassigned', 'lying', 'sitting', 'standing ', 'walking', 'running', 'cycling', 
                'nordic_walking ', 'missing_8', 'watching_tv ', 'computer_work ', 'car_driving ', 
                'ascending_stairs ', 'descending_stairs ', 'missing_14', 'missing_15', 
                'vacuuming', 'ironing', 'folding_laundry', 'house_cleaning', 'playing soccer',
                'missing_21', 'missing_22', 'missing_23', 'rope jumping'] 


if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type=float, default=0.3)
    parser.add_argument('--train-filename', type=str, default='subject101.csv')
    args, _ = parser.parse_known_args()
    
    print('Received arguments {}'.format(args))

    train_filename = args.train_filename
    print('Using train_filename: {}'.format(train_filename))
    
    input_data_path = os.path.join('/opt/ml/processing/input', train_filename)
    
    print('Reading input data from {}'.format(input_data_path))
    df = pd.read_csv(input_data_path, header=0, names=column_names, index_col=False)
    
    print('Number of elements BEFORE drop, backfill, and dropna: {}'.format(df.size))
    
    # Remove the timestamp column which only serves as our index
    df.drop('timestamp', axis=1)
    
    # Backfill heart rate column which is sparsely populated
    df_heart_rate_column = df.pop('heart_rate')
    df_heart_rate_column.fillna(value=None, method='backfill', axis=0, inplace=True)
    df.insert(2, 'heart_rate', df_heart_rate_column)
    print('Number of elements AFTER backfill: {}'.format(df.size))
    
    df.dropna(inplace=True)
    print('Number of elements AFTER dropna: {}'.format(df.size))
        
    # dump some stats
    print('DataFrame shape: {}'.format(df.shape))
    print('Num unique per column: \n{}'.format(df.nunique(axis='index')))
        
    split_ratio = args.train_test_split_ratio
    print('Splitting data into train and test sets with ratio {}'.format(split_ratio))
    X_train, X_test, y_train, y_test = train_test_split(df.drop('activity_id', axis=1), df['activity_id'], test_size=split_ratio, random_state=0)

    preprocess = make_column_transformer(
        (['heart_rate'], KBinsDiscretizer(encode='onehot-dense', n_bins=16)),
        ([ 'imu_wrist_temp', 'imu_wrist_accel16_x', 'imu_wrist_accel16_y', 'imu_wrist_accel16_z',
 'imu_wrist_accel6_x', 'imu_wrist_accel6_y', 'imu_wrist_accel6_z',
 'imu_wrist_gyro_x', 'imu_wrist_gyro_y', 'imu_wrist_gyro_z',
 'imu_wrist_magnet_x', 'imu_wrist_magnet_y', 'imu_wrist_magnet_z',
 'imu_chest_temp', 'imu_chest_accel16_x', 'imu_chest_accel16_y', 'imu_chest_accel16_z',
 'imu_chest_accel6_x', 'imu_chest_accel6_y', 'imu_chest_accel6_z',
 'imu_chest_gyro_x', 'imu_chest_gyro_y', 'imu_chest_gyro_z',
 'imu_chest_magnet_x', 'imu_chest_magnet_y', 'imu_chest_magnet_z',
 'imu_ankle_temp', 'imu_ankle_accel16_x', 'imu_ankle_accel16_y', 'imu_ankle_accel16_z',
 'imu_ankle_accel6_x', 'imu_ankle_accel6_y', 'imu_ankle_accel6_z',
 'imu_ankle_gyro_x', 'imu_ankle_gyro_y', 'imu_ankle_gyro_z',
 'imu_ankle_magnet_x', 'imu_ankle_magnet_y', 'imu_ankle_magnet_z'], 
         StandardScaler()),
        (['imu_wrist_orient1', 'imu_wrist_orient2', 'imu_wrist_orient3', 'imu_wrist_orient4',
 'imu_chest_orient1', 'imu_chest_orient2', 'imu_chest_orient3', 'imu_chest_orient4',
 'imu_ankle_orient1', 'imu_ankle_orient2', 'imu_ankle_orient3', 'imu_ankle_orient4'], 
         OneHotEncoder(sparse=False))
    )
    
    print('Running preprocessing and feature engineering transformations')
    train_features = preprocess.fit_transform(X_train)
    test_features = preprocess.transform(X_test)
    
    print('Train data shape after preprocessing: {}'.format(train_features.shape))
    print('Test data shape after preprocessing: {}'.format(test_features.shape))
    
    train_features_output_path = os.path.join('/opt/ml/processing/train', 'train_features.csv')
    train_labels_output_path = os.path.join('/opt/ml/processing/train', 'train_labels.csv')
    
    test_features_output_path = os.path.join('/opt/ml/processing/test', 'test_features.csv')
    test_labels_output_path = os.path.join('/opt/ml/processing/test', 'test_labels.csv')
    
    print('Saving training features to {}'.format(train_features_output_path))
    pd.DataFrame(train_features).to_csv(train_features_output_path, header=False, index=False)
    
    print('Saving test features to {}'.format(test_features_output_path))
    pd.DataFrame(test_features).to_csv(test_features_output_path, header=False, index=False)
    
    print('Saving training labels to {}'.format(train_labels_output_path))
    y_train.to_csv(train_labels_output_path, header=False, index=False)
    
    print('Saving test labels to {}'.format(test_labels_output_path))
    y_test.to_csv(test_labels_output_path, header=False, index=False)


Overwriting preprocessing.py


Run this script as a processing job. Use the `SKLearnProcessor.run()` method. You give the `run()` method one `ProcessingInput` where the `source` is the census dataset in Amazon S3, and the `destination` is where the script reads this data from, in this case `/opt/ml/processing/input`. These local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to. For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name/`. You also give the ProcessingOutputs values for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The `arguments` parameter in the `run()` method are command-line arguments in our `preprocessing.py` script.

In [18]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(
                        source=input_data_uri,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test_data',
                                                source='/opt/ml/processing/test')],
                      arguments=['--train-test-split-ratio', '0.3']
                     )

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'train_data':
        preprocessed_training_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'test_data':
        preprocessed_test_data = output['S3Output']['S3Uri']

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  sagemaker-scikit-learn-2020-10-01-14-58-36-580
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://octank-smartwatch-data/sagemaker-train/subject101.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-14-58-36-580/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-14-58-36-580/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-572539092864

Now inspect the output of the pre-processing job, which consists of the processed features.

In [19]:
print(preprocessed_training_data)

s3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-14-58-36-580/output/train_data


In [20]:
training_features = pd.read_csv(preprocessed_training_data + '/train_features.csv', nrows=10)
print('Training features shape: {}'.format(training_features.shape))
training_features.head(n=10)

Training features shape: (10, 67)


,0.0,0.0.1,0.0.2,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,0.0.8,0.0.9,...,1.0.3,1.0.4,1.0.5,1.0.6,1.0.7,1.0.8,1.0.9,1.0.10,1.0.11,1.0.12
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Training using the pre-processed data

We create a `SKLearn` instance, which we will use to run a training job using the training script `train.py`.  

In [21]:
from sagemaker.sklearn.estimator import SKLearn

# Change Log:
# chg #1> 
#  added framework_version='0.20.0',
#  added hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})
sklearn = SKLearn(
    entry_point='train.py',
    train_instance_type="ml.m5.4xlarge",
    framework_version='0.20.0',
    hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1},
    role=role)

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


The training script `train.py` trains a logistic regression model on the training data, and saves the model to the `/opt/ml/model` directory, which Amazon SageMaker tars and uploads into a `model.tar.gz` file into S3 at the end of the training job.

In [22]:
%%writefile train.py

import os

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

if __name__=="__main__":
    training_data_directory = '/opt/ml/input/data/train'
    train_features_data = os.path.join(training_data_directory, 'train_features.csv')
    train_labels_data = os.path.join(training_data_directory, 'train_labels.csv')
    print('Reading input data')
    X_train = pd.read_csv(train_features_data, header=None)
    y_train = pd.read_csv(train_labels_data, header=None)

    # LogisticRegression multi-class classifier
    # Change Log:
    # chg #1> added params: max_iter=1000, n_jobs=2
    # chg #2> added multi_class='ovr'; removed n_jobs
    model = LogisticRegression(class_weight='balanced', solver='lbfgs', multi_class='ovr', max_iter=200)
    print('Training LR model for multi-class classification')
    model.fit(X_train, y_train)
    model_output_directory = os.path.join('/opt/ml/model', "model.joblib")
    print('Saving model to {}'.format(model_output_directory))
    joblib.dump(model, model_output_directory)

Overwriting train.py


Run the training job using `train.py` on the preprocessed training data.

In [23]:
sklearn.fit({'train': preprocessed_training_data})
training_job_description = sklearn.jobs[-1].describe()
model_data_s3_uri = '{}{}/{}'.format(
    training_job_description['OutputDataConfig']['S3OutputPath'],
    training_job_description['TrainingJobName'],
    'output/model.tar.gz')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-01 16:30:24 Starting - Starting the training job...
2020-10-01 16:30:26 Starting - Launching requested ML instances......
2020-10-01 16:31:32 Starting - Preparing the instances for training...
2020-10-01 16:32:22 Downloading - Downloading input data......
2020-10-01 16:33:13 Training - Training image download completed. Training in progress...2020-10-01 16:33:13,950 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-10-01 16:33:13,953 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-01 16:33:13,962 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-10-01 16:33:14,226 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-10-01 16:33:14,227 sagemaker-containers INFO     Generating setup.cfg
2020-10-01 16:33:14,227 sagemaker-containers INFO     Generating MANIFEST.in
2020-10-01 16:33:14,227 sagemaker-containers INFO     Installing m

/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)

2020-10-01 16:35:55 Uploading - Uploading generated training model/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the 

## Model Evaluation

`evaluation.py` is the model evaluation script. Since the script also runs using scikit-learn as a dependency,  run this using the `SKLearnProcessor` you created previously. This script takes the trained model and the test dataset as input, and produces a JSON file containing classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.


In [29]:
%%writefile evaluation.py

import json
import os
import tarfile

import pandas as pd

from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

if __name__=="__main__":
    model_path = os.path.join('/opt/ml/processing/model', 'model.tar.gz')
    print('Extracting model from path: {}'.format(model_path))
    with tarfile.open(model_path) as tar:
        tar.extractall(path='.')
    print('Loading model')
    model = joblib.load('model.joblib')

    print('Loading test input data')
    test_features_data = os.path.join('/opt/ml/processing/test', 'test_features.csv')
    test_labels_data = os.path.join('/opt/ml/processing/test', 'test_labels.csv')

    X_test = pd.read_csv(test_features_data, header=None)
    y_test = pd.read_csv(test_labels_data, header=None)
    predictions = model.predict(X_test)

    print('Creating classification evaluation report')
    report_dict = classification_report(y_test, predictions, output_dict=True)
    report_dict['accuracy'] = accuracy_score(y_test, predictions)

    print('Classification report:\n{}'.format(report_dict))

    evaluation_output_path = os.path.join('/opt/ml/processing/evaluation', 'evaluation.json')
    print('Saving classification report to {}'.format(evaluation_output_path))

    with open(evaluation_output_path, 'w') as f:
        f.write(json.dumps(report_dict))

Overwriting evaluation.py


In [27]:
import json
from sagemaker.s3 import S3Downloader

sklearn_processor.run(code='evaluation.py',
                      inputs=[ProcessingInput(
                                  source=model_data_s3_uri,
                                  destination='/opt/ml/processing/model'),
                              ProcessingInput(
                                  source=preprocessed_test_data,
                                  destination='/opt/ml/processing/test')],
                      outputs=[ProcessingOutput(output_name='evaluation',
                                  source='/opt/ml/processing/evaluation')]
                     )                    
evaluation_job_description = sklearn_processor.jobs[-1].describe()

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  sagemaker-scikit-learn-2020-10-01-17-02-14-357
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-16-30-23-420/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-14-58-36-580/output/test_data', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-scikit-learn-2020-10-01-17-02-14-357/input/code/evaluation.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated',

Now retrieve the file `evaluation.json` from Amazon S3, which contains the evaluation report.

In [28]:
evaluation_output_config = evaluation_job_description['ProcessingOutputConfig']
for output in evaluation_output_config['Outputs']:
    if output['OutputName'] == 'evaluation':
        evaluation_s3_uri = output['S3Output']['S3Uri'] + '/evaluation.json'
        break

evaluation_output = S3Downloader.read_file(evaluation_s3_uri)
evaluation_output_dict = json.loads(evaluation_output)
print(json.dumps(evaluation_output_dict, sort_keys=True, indent=4))

{
    "0": {
        "f1-score": 0.840174490449179,
        "precision": 0.9562800786280756,
        "recall": 0.7492100581503413,
        "support": 37661
    },
    "1": {
        "f1-score": 0.9757814364113577,
        "precision": 0.9604274307186472,
        "recall": 0.9916343355965083,
        "support": 8248
    },
    "12": {
        "f1-score": 0.8411393614987575,
        "precision": 0.7670850767085077,
        "recall": 0.9310198899703767,
        "support": 4726
    },
    "13": {
        "f1-score": 0.7931130758492322,
        "precision": 0.6834001603849238,
        "recall": 0.944789356984479,
        "support": 4510
    },
    "16": {
        "f1-score": 0.9673423423423423,
        "precision": 0.9543119011248438,
        "recall": 0.9807335521621235,
        "support": 7007
    },
    "17": {
        "f1-score": 0.9093167701863354,
        "precision": 0.8380332610267535,
        "recall": 0.993854509075318,
        "support": 6997
    },
    "2": {
        "f1-score":